# Vocal Removal - Open-Unmix

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
os.chdir('/content/drive/My Drive/umx')

Mounted at /content/drive


In [ ]:
!pip install openunmix

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 847.0 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from openunmix import predict

In [ ]:
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## Evaluation

In [ ]:
!pip install musdb
!pip install museval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 1.9 MB/s eta 0:00:00


Open-Unmix has a feature where it can predict a particular track and everything besides that track (a residual).

In [ ]:
import musdb
import museval

def estimate_and_evaluate(track):
    predictions = predict.separate(
        torch.as_tensor(track.audio).float(),
        rate=track.rate,
        targets=['vocals'],
        residual=True,
        device=device
    )

    estimates = {
        'vocals': predictions['vocals'].detach().cpu().numpy()[0].T,
        'accompaniment': predictions['residual'].detach().cpu().numpy()[0].T
    }

    scores = museval.eval_mus_track(
        track, estimates, output_dir="output"
    )

    return scores

In [ ]:
mus = musdb.DB(download=True, subsets="test")

100%|██████████| 140M/140M [00:00<00:00, 222MB/s]


In [ ]:
import re

def parse_audio_metrics(audio_metrics):
    first_line = audio_metrics.split('\n')[0]
    pattern = r"(SDR|SIR|ISR|SAR):\s*([-+]?\d*\.\d+|\d+)"
    metrics = dict(re.findall(pattern, first_line))
    metrics = {key: float(value) for key, value in metrics.items()}
    return metrics

In [ ]:
sdr = []
sir = []
isr = []
sar = []
n = len(mus)
for i, track in enumerate(mus):
  print(f"Track {i+1} of {n}")
  scores = estimate_and_evaluate(track)
  metrics = parse_audio_metrics(str(scores))
  sdr.append(metrics['SDR'])
  sir.append(metrics['SIR'])
  isr.append(metrics['ISR'])
  sar.append(metrics['SAR'])

Track 1 of 50
Track 2 of 50
Track 3 of 50
Track 4 of 50
Track 5 of 50
Track 6 of 50
Track 7 of 50
Track 8 of 50
Track 9 of 50
Track 10 of 50
Track 11 of 50
Track 12 of 50
Track 13 of 50
Track 14 of 50
Track 15 of 50
Track 16 of 50
Track 17 of 50
Track 18 of 50
Track 19 of 50
Track 20 of 50
Track 21 of 50
Track 22 of 50
Track 23 of 50
Track 24 of 50
Track 25 of 50
Track 26 of 50
Track 27 of 50
Track 28 of 50
Track 29 of 50
Track 30 of 50
Track 31 of 50
Track 32 of 50
Track 33 of 50
Track 34 of 50
Track 35 of 50
Track 36 of 50
Track 37 of 50
Track 38 of 50
Track 39 of 50
Track 40 of 50
Track 41 of 50
Track 42 of 50
Track 43 of 50
Track 44 of 50
Track 45 of 50
Track 46 of 50
Track 47 of 50
Track 48 of 50
Track 49 of 50
Track 50 of 50


In [ ]:
for track in mus:
  print(f"{track.name}")

In [ ]:
for arr in [sdr, sir, isr, sar]:
  for s in arr:
    print(s)
  print()

9.553
7.503
9.949
4.898
5.471
9.593
4.533
7.572
3.272
7.098
13.783
8.641
9.844
7.917
6.97
4.295
5.818
5.644
9.593
8.495
7.034
12.766
7.267
12.796
-0.573
12.827
7.798
-4.458
0.028
8.158
5.707
8.182
7.894
8.443
10.785
8.763
3.71
4.135
6.055
6.999
4.168
3.414
8.07
7.809
9.223
3.801
5.829
9.031
5.601
7.79

16.578
12.038
16.386
9.671
12.959
15.193
7.936
10.612
9.104
11.034
18.932
11.445
14.993
11.998
9.108
7.547
8.388
12.144
13.129
15.147
13.029
19.104
12.238
19.06
-1.306
21.901
13.595
-24.36
-9.928
17.142
8.776
9.93
12.49
17.154
15.687
13.889
5.057
6.602
9.051
15.204
7.079
9.311
12.673
13.339
15.402
8.398
11.46
10.552
11.215
13.891

13.422
17.856
16.157
16.157
10.095
19.462
8.789
13.631
4.569
15.128
23.885
21.172
23.019
16.617
15.781
12.105
12.532
9.382
18.981
13.721
13.321
23.182
14.044
20.684
7.19
23.618
11.554
0.01
0.054
15.629
15.203
22.398
16.179
18.109
20.619
13.679
5.99
12.963
13.128
9.267
10.475
6.017
16.732
17.234
21.417
8.051
10.507
18.8
12.105
15.454

12.877
9.494
10.498
6.559
6